# Getting the embeddings

> This notebook gets the embeddings (or latent space) from a multivariate time series 
given by a encoder (e.g., autoencoder)

In [1]:
model_patch_size = 8
verbose = 0
reset_kernel = False

In [2]:
from dvats.all import *
from tsai.data.preparation import SlidingWindow
from fastcore.all import *
import wandb
wandb_api = wandb.Api()
from yaml import load, FullLoader
import dvats.utils as ut


Octave is ready <oct2py.core.Oct2Py object at 0x7fd814d83370>

















In [3]:
import torch
torch.cuda.set_device(1)

## Config parameters
> Configuration parameters are obtained from 'config\03-embeddings.yaml'

### Get configuration artifact

In [4]:
config, job_type = get_artifact_config_embeddings(verbose = 0)

In [5]:
dvats.config.show_attrdict(config)

use_wandb: True
wandb_group: embeddings
wandb_entity: mi-santamaria
wandb_project: deepvats
enc_artifact: mi-santamaria/deepvats/moment-small-embedding:latest
input_ar: None
cpu: False


### Show configuration artifact

In [6]:
for key, value in config.items():
    print(f"{key}: {value}")

use_wandb: True
wandb_group: embeddings
wandb_entity: mi-santamaria
wandb_project: deepvats
enc_artifact: mi-santamaria/deepvats/moment-small-embedding:latest
input_ar: None
cpu: False


## Build W&B artifact

In [7]:
import os
path = os.path.expanduser("~/work/nbs_pipeline/")
name="03a_embeddings"
os.environ["WANDB_NOTEBOOK_NAME"] = path+name+".ipynb"
runname=name
print("runname: "+runname)

runname: 03a_embeddings


In [8]:
run = wandb.init(
    entity      = config.wandb_entity,
    project     = config.wandb_project if config.use_wandb else 'work-nbs', 
    group       = config.wandb_group,
    job_type    = job_type,
    mode        = 'online' if config.use_wandb else 'disabled',
    anonymous   = 'never' if config.use_wandb else 'must',
    config      = config,
    resume      = 'allow',
    name        = runname
)

wandb: Currently logged in as: mi-santamaria. Use `wandb login --relogin` to force relogin


## Get trained model artifact

### Build artifact selector
> Botch to use artifacts offline

In [9]:
artifacts_gettr = run.use_artifact if config.use_wandb else wandb_api.artifact

### Get the model from W&B
> Restore the encoder model and its associated configuration

In [10]:
enc_artifact = artifacts_gettr(config.enc_artifact, type='learner')

In [11]:
# TODO: This only works when you run it two timeS! WTF?
try:
    enc_learner = enc_artifact.to_obj()
except:
    enc_learner = enc_artifact.to_obj()

wandb: Downloading large artifact moment-small-embedding:latest, 144.63MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.4


## Get dataset artifact from W&B
### Restore the dataset artifact used for training the encoder. 
> Even if we do not compute the dimensionality reduction over this dataset, we need to know the metadata of the encoder training set, to check that it matches with the dataset that we want to reduce.

In [12]:
enc_run = enc_artifact.logged_by()
enc_artifact_train = artifacts_gettr(enc_run.config['train_artifact'], type='dataset')
enc_artifact_train.name

'toy:v2'

In [13]:
dvats.config.show_attrdict(enc_run.config)

r: 0.71
w: 30
freq: 1s
alias: toy
epochs: 100
mvp_ws: [10, 30]
stride: 1
time_col: None
data_cols: []
mask_sync: False
use_wandb: True
batch_size: 32
csv_config: {}
data_fpath: ~/data/toy.csv
valid_size: 0.2
mask_future: False
wandb_group: None
analysis_mode: online
artifact_name: toy
mask_stateful: True
norm_by_sample: False
train_artifact: mi-santamaria/deepvats/toy:latest
valid_artifact: None
norm_use_single_batch: False
norm_use_by_single_batch: [False]


### Specify the dataset artifact that we want to get the embeddings from
> If no artifact is defined, the artifact to reduce will be the one used for validate the encoder.

In [14]:
enc_run.config['batch_size']

32

In [15]:
input_ar_name = ifnone(
    config.input_ar, 
    f'{enc_artifact_train.entity}/{enc_artifact_train.project}/{enc_artifact_train.name}'
)
wandb.config.update({'input_ar': input_ar_name}, allow_val_change=True)
input_ar = artifacts_gettr(input_ar_name)
input_ar.name

'toy:v2'

In [16]:
df = input_ar.to_df()
df.head()

wandb:   1 of 1 files downloaded.  


,T3,T2,T1
1970-01-01 00:00:00,0.741822,0.637180,0.565117
1970-01-01 00:00:01,0.739731,0.629415,0.493513
1970-01-01 00:00:02,0.718757,0.539220,0.469350
1970-01-01 00:00:03,0.730169,0.577670,0.444100
1970-01-01 00:00:04,0.752406,0.570180,0.373008


In [17]:
df.shape

(550, 3)

In [18]:
enc_input, _ = SlidingWindow(window_len=enc_run.config['w'], 
                             stride=enc_run.config['stride'], 
                             get_y=[])(df)
enc_input.shape

(521, 3, 30)

In [19]:
timer = ut.Time()
timer.start()

1729181534.4783401

In [20]:
config.enc_artifact

'mi-santamaria/deepvats/moment-small-embedding:latest'

In [21]:
print(enc_learner)

MOMENTPipeline(
  (normalizer): RevIN()
  (tokenizer): Patching()
  (patch_embedding): PatchEmbedding(
    (value_embedding): Linear(in_features=8, out_features=512, bias=False)
    (position_embedding): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (

In [22]:
enc_learn_class = str(enc_learner.__class__)[8:-2]

match enc_learn_class:
    case "momentfm.models.moment.MOMENTPipeline":
        get_embs_kwargs = {
            "cpu": config.cpu,
            "to_numpy": True,
            "verbose": 1
        }
    case "fastai.learner.Learner":
        get_embs_kwargs = {
            "stride": enc_run.config['stride'],
            "cpu": config.cpu,
            "to_numpy": True,
            "batch_size": enc_run.config['batch_size'],
            "average_seq_dim": True,
            "verbose": 1
        }
    case "uni2ts.model.moirai.module.MoiraiModule":
        get_embs_kwargs = {
            "cpu": config.cpu,
            "to_numpy": True,
            "batch_size": enc_run.config['batch_size'],
            "average_seq_dim": True,
            "verbose": 1,
            "patch_size": 8, #Modificar en config (añadir en base.yml & modificar lectura a "si existe, añadir"),
            "size": "small", #Modificar en config (añadir en base.yml & modificar lectura a "si existe, añadir"),
            "time": True
        }
    case _:
        print(f"Model embeddings implementation is not yet implemented for {enc_learn_class}.")

In [23]:
enc_learn_class

'momentfm.models.moment.MOMENTPipeline'

In [24]:
enc_learn_class = str(enc_learner.__class__)[8:-2]
enc_learn_class

'momentfm.models.moment.MOMENTPipeline'

In [25]:
match enc_learn_class:
    case "momentfm.models.moment.MOMENTPipeline":
        get_embs_kwargs = {
            "batch_size": enc_input.shape[0],
            "cpu": config.cpu,
            "to_numpy": True,
            "verbose": 1,
            "padd_step":2
        }
    case "fastai.learner.Learner":
        get_embs_kwargs = {
            "stride": enc_run.config['stride'],
            "cpu": config.cpu,
            "to_numpy": True,
            "batch_size": enc_run.config['batch_size'],
            "average_seq_dim": True,
            "verbose": 1
        }
    case "uni2ts.model.moirai.module.MoiraiModule":
        get_embs_kwargs = {
            "cpu": config.cpu,
            "to_numpy": True,
            "batch_size": enc_run.config['batch_size'],
            "average_seq_dim": True,
            "verbose": 2,
            "patch_size": model_patch_size, #Modificar en config (añadir en base.yml & modificar lectura a "si existe, añadir"),
            "time": True
        }
    case _:
        print(f"Model embeddings implementation is not yet implemented for {enc_learn_class}.")
print(f"Enc learn class {enc_learn_class}\nkwargs: {get_embs_kwargs}")

Enc learn class momentfm.models.moment.MOMENTPipeline
kwargs: {'batch_size': 521, 'cpu': False, 'to_numpy': True, 'verbose': 1, 'padd_step': 2}


In [26]:
import dvats.utils as ut
timer = ut.Time()

In [34]:
timer.start()
fine_tune_moment_(
    X               = enc_input,
    enc_learn       = enc_learner, 
    stride          =  1,      
    batch_size      = 32,
    cpu             = True, 
    average_seq_dim = True, 
    to_numpy        = True, 
    verbose         = 1, 
    time_flag       = False,
    #n_windows       = 32,
    percent         = 0.3,
    padd_step       = 100,
    max_trials      = 3
)
timer.end()
timer.show()

Random windows | n_windows: 157
windows ~ torch.Size([157, 3, 30])
loss 0: 0.2628089487552643
loss 32: 0.22773772478103638
loss 64: 0.22906725108623505
loss 96: 0.23963670432567596
loss 128: 0.23882727324962616
[] Start: 1729181564.535719 | End: 1729181582.9802225 | Duration: 18.44450354576111 seconds


18.44450354576111

In [30]:
embs = get_enc_embs_set_stride_set_batch_size(
    X          = enc_input, 
    enc_learn  = enc_learner, 
    stride     = enc_run.config['stride'],
    **get_embs_kwargs
)

--> get_enc_embs_set_stride_set_batch_size
get_enc_embs_set_stride_set_batch_size | Moment | True
get_acts_moment | Trial 1 | x_enc ~ torch.Size([521, 3, 30])
get_acts_moment | Trial 1 | About to pad X (encoder input) | exception The size of tensor a (24) must match the size of tensor b (30) at non-singleton dimension 1 | padd step: 2
get_acts_moment | Trial 1 | y ~ torch.Size([521, 3, 30])
------------------- Trial 1 | Padd -----------------
get_acts_moment | Trial 2 | x_enc ~ torch.Size([521, 3, 32])
get_acts_moment | Trial 2 | embs ~ torch.Size([521, 3, 4, 512])
--> get acts | acts indices: None
get acts | Moving to gpu
get acts | Add hooks
get acts | --> Run forward
get_enc_embs_moment | About to pad X (encoder input) | exception The size of tensor a (24) must match the size of tensor b (30) at non-singleton dimension 1 | padd step: 2
--> get acts | acts indices: None
get acts | Moving to gpu
get acts | Add hooks
get acts | --> Run forward
get acts | Run forward -->
get acts -->
ge

In [31]:
timer.end()
timer.show()

[] Start: 1729181534.5587578 | End: 1729181553.5116665 | Duration: 18.952908754348755 seconds


18.952908754348755

In [32]:
#| export
if verbose > 0: print("Execution ended")
from dvats.imports import beep
beep(1)
beep(1)
beep(1)
beep(1)
beep(1)

In [33]:
#| hide
if reset_kernel:
    import os
    os._exit(00)